In [19]:
from math import *
from collections import defaultdict
from typing import List

class Breakpoint:
    dest_start : int
    source_start : int
    length : int

    def __repr__(self) -> str:
        return f"[{self.source_start}, {self.length}, {self.dest_start}]"

class Mapper:
    breakpoints : List[Breakpoint]

    def __init__(self) -> None:
        self.breakpoints = []

    def sort(self) -> None:
        self.breakpoints.sort(key = lambda b : b.source_start)

    def eval(self, x : int) -> int:
        if x < self.breakpoints[0].source_start:
            return x
        
        for breakpoint in self.breakpoints[::-1]:
            if breakpoint.source_start <= x:
                if x < breakpoint.source_start + breakpoint.length:
                    return breakpoint.dest_start + (x - breakpoint.source_start)
                return x
    
    def __repr__(self) -> str:
        return f"{self.breakpoints}"



with open('input.txt', 'r') as f:
    lines = [l[:-1] for l in f.readlines()]

seeds = [int(i) for i in lines[0][7:].split(' ')]


mappers : List[Mapper] = []
current_mapper = -1
for line in lines[2:]:
    if line == "":
        mappers[current_mapper].sort()
        continue
    if line[0] in "abcdefghijklmnopqrstuvwxyz":
        mappers.append(Mapper())
        current_mapper += 1
        continue
    b_cur = Breakpoint()
    b_cur.dest_start, b_cur.source_start, b_cur.length = [int(i) for i in line.split(' ')]
    mappers[current_mapper].breakpoints.append(b_cur)
mappers[current_mapper].sort()

locations = []
for seed in seeds:
    for mapper in mappers:
        seed = mapper.eval(seed)
    locations.append(seed)

print(min(locations))

51580674


In [1]:
from math import *
from collections import defaultdict
from typing import List

class Breakpoint:
    dest_start : int
    source_start : int
    length : int

    def source_end(self) -> int:
        return self.source_start + self.length

    def __repr__(self) -> str:
        return f"[{self.source_start}, {self.length}, {self.dest_start}]"
    
class Interval:
    left : int  # incl
    right : int # excl
    offset : int

    def __init__(self, left, right, offset) -> None:
        self.left = left
        self.right = right
        self.offset = offset
    
    def __repr__(self) -> str:
        return f"<{self.left}, {self.right}> ({self.offset})"
    

def overlap(a : Interval, b : Interval) -> [Interval]:
    if a.right <= b.left:
        return []
    
    if a.left <= b.left and a.right <= b.right:
        return [Interval(b.left, a.right, a.offset + b.offset)]
    
    if a.left <= b.left and b.right <= a.right:
        return [Interval(b.left, b.right, a.offset + b.offset)]
    
    if b.left <= a.left and a.right <= b.right:
        return [Interval(a.left, a.right, a.offset + b.offset)]
    
    if b.left <= a.left and b.right <= a.right:
        return [Interval(a.left, b.right, a.offset + b.offset)]
    
    if b.right <= a.left:
        return []

class Mapper:
    breakpoints : List[Breakpoint]

    def __init__(self) -> None:
        self.breakpoints = []

    def sort(self) -> None:
        self.breakpoints.sort(key = lambda b : b.source_start)

    
    def eval_range(self, intervals : List[Interval]) -> List[Interval]:
        res = []
        for interval in intervals:
            cuts = self.cut(interval)
            cuts =  [i for i in cuts if i.left < i.right]
            res += cuts
        return res

    
    def cut(self, interval : Interval) -> List[Interval]:
        if interval.right <= self.breakpoints[0].source_start:
            return [interval]
        
        if self.breakpoints[-1].source_end() <= interval.left:
            return [interval]
        
        result = []
        in_interval = False
        if interval.left <= self.breakpoints[0].source_start:
            in_interval = True
            result.append(Interval(interval.left, self.breakpoints[0].source_start, interval.offset))

        for b_cur, b_next in zip(self.breakpoints[:-1], self.breakpoints[1:]):
            on_interval = Interval(b_cur.source_start, b_cur.source_end(), b_cur.dest_start - b_cur.source_start)
            o = overlap(interval, on_interval)
            if len(o) > 0:
                in_interval = True
                result.append(o[0])
            if len(o) == 0 and in_interval:
                return result
            
            off_interval = Interval(b_cur.source_end(), b_next.source_start, 0)
            o = overlap(interval, off_interval)
            if len(o) > 0:
                in_interval = True
                result.append(o[0])
            if len(o) == 0 and in_interval:
                return result
            
        b_last = self.breakpoints[-1]
        on_interval = Interval(b_last.source_start, b_last.source_end(), b_last.dest_start - b_last.source_start)
        o = overlap(interval, on_interval)
        if len(o) > 0:
            result.append(o[0])
        if len(o) == 0 and in_interval:
            return result
        
        if b_last.source_end() < interval.right:
            result.append(Interval(b_last.source_end(), interval.right, interval.offset))
        
        return result





    def __repr__(self) -> str:
        return f"{self.breakpoints}"


def merge(intervals : List[Interval]) -> List[Interval]:
    # only when all offsets are 0 !
    if len(intervals) < 2:
        return intervals
    
    res = []
    current_left = intervals[0].left
    for a, b in zip(intervals[:-1], intervals[1:]):
        if a.right != b.left:
            res.append(Interval(current_left, a.right, 0))
            current_left = b.left
    
    res.append(Interval(current_left, intervals[-1].right, 0))

    return res

    


with open('input.txt', 'r') as f:
    lines = [l[:-1] for l in f.readlines()]

seed_numbers = [int(i) for i in lines[0][7:].split(' ')]
seeds = [Interval(i,i+j,0) for i,j in zip(seed_numbers[::2], seed_numbers[1::2])]


mappers : List[Mapper] = []
current_mapper = -1
for line in lines[2:]:
    if line == "":
        mappers[current_mapper].sort()
        continue
    if line[0] in "abcdefghijklmnopqrstuvwxyz":
        mappers.append(Mapper())
        current_mapper += 1
        continue
    b = Breakpoint()
    b.dest_start, b.source_start, b.length = [int(i) for i in line.split(' ')]
    mappers[current_mapper].breakpoints.append(b)
mappers[current_mapper].sort()

debug = False

# seeds = [Interval(10, 20, 0), Interval(40, 60, 0)]
for mapper in mappers:
    mapped = mapper.eval_range(seeds)
    debug and print(mapped)
    applied = [Interval(i.left + i.offset, i.right + i.offset, 0) for i in mapped]
    debug and print(applied)
    sorted_list = sorted(applied, key=lambda i : i.left)
    debug and print(sorted_list)
    merged = merge(sorted_list)
    debug and print(merged)
    seeds = merged


print(merged[0].left)



99751240
